In [ ]:


!pip install  transformers
!pip install langchain
!pip install -U langchain-community
!pip install unstructured
!pip install "unstructured[pdf]"
!apt-get install -y poppler-utils
!pip install LangChain -q
!pip install -U langchain-community -q
!pip install langchain transformers
!pip install chromadb
!pip install sentence-transformers
!pip install chromadb
!pip install langchain-nvidia-ai-endpoints
!pip install PyPDF2
!pip install rank-bm25 sentence-transformers
!pip install chromadb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 0 B/186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Selecting previously unselected package poppler-utils.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [1]:
import os
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader


In [2]:
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader


# Define the text splitter
text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust chunk size based on content length
    chunk_overlap=0,  # No overlap needed
    separators=["Page number"]  # Split at answer or next question
)

# Load and preprocess the data
data_loader = TextLoader("/content/الجوانب_العملية_فى_الاجراءات_الجنائية_2016.txt")





In [3]:
# Load documents from the specified directory
data = data_loader.load_and_split(text_splitter=text_splitter_recursive) # Load documents
print(f"Documents : {len(data)}.")

Documents : 345.


In [4]:
# Remove "Page number " from each chunk using regex
for i in range(len(data)):
  data[i].page_content = re.sub(r"Page number: \d+", "", data[i].page_content)

In [5]:
import pandas as pd
df = pd.read_csv('/content/الجوانب_العملية_فى_الاجراءات_الجنائية_2016.csv')


In [6]:

import re

df['context'] = df['context'].apply(lambda x: re.sub(r'Page number: \d+', '', str(x)))

In [ ]:
# from langchain.document_loaders.csv_loader import CSVLoader

# loader_csv = CSVLoader(file_path="/content/الجوانب_العملية_فى_الاجراءات_الجنائية_2016.csv")
# data_csv = loader_csv.load()

In [7]:
df.loc[0,"question"]

'ما هي إجراءات الطالب بعد اكتشافه للتزوير في دعوى الإخلاء؟'

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the base model
base_model = SentenceTransformer('intfloat/multilingual-e5-large')

In [28]:
from sentence_transformers import SentenceTransformer
import chromadb


path = "/content/DB"
client = chromadb.PersistentClient(path=path)

db_file = client.get_or_create_collection(name='Book',
                                          metadata={"hnsw:space": "cosine", "dimension": 1024})


def embed(texts):
    return base_model.encode(texts)

i = 0
for chunk in data:
    embeddings = embed([chunk.page_content])  # Get embeddings using the base model
    db_file.add(
        documents=[chunk.page_content],
        ids=[f"chunk_{i}"],
        embeddings=embeddings  # Pass the embeddings directly
    )
    i += 1


In [29]:
db_file.get(ids=["chunk_2"])

{'ids': ['chunk_2'],
 'embeddings': None,
 'documents': ['\n\n- ٤ -\nالقضية (٢)\nأمر النائب العام أحد أعضاء النيابة العامة برفع الدعوى الجنائية ولكن عضو\nالنيابة، لم يستجب لذلك وأصدر قرارا بأن لا وجه لإقامة الدعوى، فما رأيك في هذا\nالإجراء؟\nالإجابة\nالمسائل القانونية التي تثيرها القضية : تتعلق أيضا هذه القضية بخاصية التبعية\nالتدريجية كإحدى خصائص النيابة العامة.\nالأسانيد القانونية : يتبع أعضاء النيابة - عدا نيابة النقض - النائب العام تبعية إدارية\nوقضائية. فهو يشرف إداريا على جميع أعضاء النيابة. وهو الأصيل في تحريك ومباشرة\nالدعوى الجنائية. بمعنى أن باقي أعضاء النيابة يعتبرون وكلاء عنه يستعملون الدعوى\nباسمه ويستمدون منه سلطاتهم.\nومن أجل ذلك يكون للنائب العام أن يصدر ما يراه من تعليمات الى أعضاء\nالنيابة بشأن إقامة أو مباشرة الدعوى العمومية دون التحقيق فيها، وأن يأمر بحفظ\nالتحقيق أو بأن لا وجه لإقامة الدعوى العمومية. ويترتب على مخالفة أعضاء\nالنيابة العامة لهذه التعليمات والأوامر بطلان تصرفاتهم، لأن في ذلك خروج عن\nحدود الوكالة الممنوحة لهم من النائب العام، فيكون تصرفه صادرا من غير

In [30]:
def retrieve_documents_embeddings(query_embedding, k=10):
    query_embedding_list = query_embedding.tolist()

    results = db_file.query(
        query_embeddings=[query_embedding_list],

        n_results=k)
    return results['documents'][0]

In [31]:
def reciprocal_rank(retrieved_docs, ground_truth, k):
    try:
        rank = retrieved_docs.index(ground_truth) + 1
        return 1.0 / rank if rank <= k else 0.0
    except ValueError:
        return 0.0

In [32]:
def hit_rate(retrieved_docs, ground_truth, k):
    return 1.0 if ground_truth in retrieved_docs[:k] else 0.0

In [33]:
from sklearn.model_selection import train_test_split



train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

print(f"Training DataFrame size: {train_data.shape}")
print(f"Testing DataFrame size: {test_data.shape}")


Training DataFrame size: (834, 3)
Testing DataFrame size: (209, 3)


In [36]:
import numpy as np
def validate_embedding_model(validation_data, base_model, k=10):
    hit_rates = []
    reciprocal_ranks = []

    for _, row in validation_data.iterrows():  # Iterate over rows of the DataFrame
        question = row['question']
        ground_truth = row['context']
        answer = row['answer']

        # Generate embedding for the question
        question_embedding = base_model.encode(question)
        #print(" ### Question embedding ### :",question)
        #print(" ### Ground Truth ### :",ground_truth)

        #print(" ### Answer ### :",answer)

        # Retrieve documents using the embedding
        retrieved_docs = retrieve_documents_embeddings(question_embedding, k)
        #print("  ### Retrieved Documents #### :")
        #for idx, doc in enumerate(retrieved_docs, 1):  # Start enumeration at 1
         #   print(f"\nDocument {idx}:\n{'-' * 40}\n{doc}\n{'-' * 40}")


        # Calculate metrics
        hr = hit_rate(retrieved_docs, ground_truth, k)
        rr = reciprocal_rank(retrieved_docs, ground_truth, k)

        hit_rates.append(hr)
        reciprocal_ranks.append(rr)

    # Calculate average metrics
    avg_hit_rate = np.mean(hit_rates)
    avg_reciprocal_rank = np.mean(reciprocal_ranks)

    return {
        'average_hit_rate': avg_hit_rate,
        'average_reciprocal_rank': avg_reciprocal_rank
    }

# Call the function with the corrected implementation
results = validate_embedding_model(test_data, base_model)
print(f"Average Hit Rate @10: {results['average_hit_rate']}")
print(f"Mean Reciprocal Rank @10: {results['average_reciprocal_rank']}")


Average Hit Rate @10: 0.5598086124401914
Mean Reciprocal Rank @10: 0.3997531708058024


In [37]:
import random
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.utils import clip_grad_norm_

In [55]:
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader


# Define the text splitter
text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust chunk size based on content length
    chunk_overlap=0,  # No overlap needed
    separators=["Page number"]  # Split at answer or next question
)

# Load and preprocess the data
data_loader_negative = TextLoader("/content/Negative.txt")


# Load NVIDIA 10K Document
# Will be used for random negative sampling
#nvidia_loader = PyPDFLoader("/Users/alucek/Documents/Jupyter_Notebooks/ft_emb/data/nvidia_10k.pdf")
data_negative_pages = data_loader_negative.load()

negative_docs = ""
for i in range(len(data_negative_pages)):
    #data_negative_pages[i].page_content = re.sub(r"Page number: \d+", "", data_negative_pages[i].page_content)
    negative_docs += data_negative_pages[i].page_content

negative_chunk = text_splitter_recursive.split_text(negative_docs)
negative_chunk = negative_chunk[1:278]

for i in range(len(negative_chunk)):
  negative_chunk[i] = re.sub(r"Page number: \d+", "", negative_chunk[i])




In [56]:
print(f"Documents : {negative_chunk[2]}.")

Documents : 

١
علم النفس الإكلينيكي العلاجي
الفصل الأول
علم النفس الإكلينيكي العلاجي
(الأسس النظرية والممارسة العملية )
الأهداف:
بعد دراسة هذا الفصل؛ ينبغي أن يكون الدارس قادرًا على أن:
١ - يعرف المقصود بعلم النفس الإكلينيكي العلاجي.
٢ - يعرف تاريخ علم النفس الإكلينيكي العلاجي وتطوره.
٣- يكتشف العلاقة بين علم النفس الإكلينيكي العلاجي وفروع علم النفس
الأخرى الأساسية والتطبيقية.
٤ - يميز الفروق بين أساليب العلاج النفسي التقليدي والحديث.
٥- يتعرف على التوجهات الحديثة في العلاج النفسي.
٦ - يعرف متطلبات ممارسة العلاج النفسي.
٧- يعرف المعايير الأخلاقية لممارسة العلاج النفسي.
العناصر :
١) تعريف علم النفس الإكلينيكي العلاجي.
٢) تاريخ علم النفس الإكلينيكي العلاجي عبر ثلاث مراحل:
أ- التحول من العلاج بالسحر والشعوذة إلى العلاج العضوي.
ب-التحول من العلاج العضوي إلى التحليل النفسي
ج- التحول من الفرويدية إلى السلوكية.
٣) علاقة علم النفس الإكلينيكي العلاجي بفروع علم النفس الأخرى.


In [57]:
def random_negative():
    random_sample = random.choice(negative_chunk)
    return random_sample

In [58]:
print(random_negative())



علم النفس الإكلينيكي العلاجي
١٣٨
أسئلة على الفصل السادس
[2]
س١- عرف المقصود بالتسكين المنظم والأسس النظرية له .
س٢ - اشرح خطوات العلاج بالتسكين المنظم .
س٣- اشرح خطوات الاسترخاء العضلى.
س٤- تكلم عن خطوات اعداد مدرج القلق مع التمثيل .
س٥- أذكر محددات فاعلية العلاج بالتسكين المنظم.
س٦- عرف المقصود بأسلوب الغمر واشرح خطواته العلاجية .
س٧- قارن بين أسلوب التسكين المنظم والغمر.


In [59]:
class LinearAdapter(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        return self.linear(x)

In [68]:
class TripletDataset(Dataset):
    def __init__(self, data, base_model, negative_sampler):
        self.data = data
        self.base_model = base_model
        self.negative_sampler = negative_sampler

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
       # print(f"Accessing index: {idx}")

        row = self.data.iloc[idx]
        query = row['question']
        positive = row['context']
        negative = self.negative_sampler()



        query_emb = self.base_model.encode(query, convert_to_tensor=True)
        positive_emb = self.base_model.encode(positive, convert_to_tensor=True)
        negative_emb = self.base_model.encode(negative, convert_to_tensor=True)

        return query_emb, positive_emb, negative_emb

In [69]:
def get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps)))
    return LambdaLR(optimizer, lr_lambda)

def train_linear_adapter(base_model, train_data, negative_sampler, num_epochs=10, batch_size=32,
                         learning_rate=2e-5, warmup_steps=100, max_grad_norm=1.0, margin=1.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize the LinearAdapter
    adapter = LinearAdapter(base_model.get_sentence_embedding_dimension()).to(device)

    # Define loss function and optimizer
    triplet_loss = nn.TripletMarginLoss(margin=margin, p=2)
    optimizer = AdamW(adapter.parameters(), lr=learning_rate)

    # Create dataset and dataloader
    dataset = TripletDataset(train_data, base_model, negative_sampler)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Calculate total number of training steps
    total_steps = len(dataloader) * num_epochs

    # Create learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            query_emb, positive_emb, negative_emb = [x.to(device) for x in batch]

            # Forward pass
            adapted_query_emb = adapter(query_emb)

            # Compute loss
            loss = triplet_loss(adapted_query_emb, positive_emb, negative_emb)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping
            clip_grad_norm_(adapter.parameters(), max_grad_norm)

            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

    return adapter

In [ ]:
# Define the kwargs dictionary
adapter_kwargs = {
    'num_epochs': 30,
    'batch_size': 32,
    'learning_rate': 0.003,
    'warmup_steps': 100,
    'max_grad_norm': 1.0,
    'margin': 1.0
}
#train_data = train_data.reset_index(drop=True)

# Train the adapter using the kwargs dictionary
trained_adapter = train_linear_adapter(base_model, train_data, random_negative, **adapter_kwargs)

Epoch 1/30, Loss: 0.7904
Epoch 2/30, Loss: 0.6533
Epoch 3/30, Loss: 0.6447
Epoch 4/30, Loss: 0.6368
Epoch 5/30, Loss: 0.6274
Epoch 6/30, Loss: 0.6140
Epoch 7/30, Loss: 0.6097
Epoch 8/30, Loss: 0.6068
Epoch 9/30, Loss: 0.5991
Epoch 10/30, Loss: 0.5953
Epoch 11/30, Loss: 0.5913
Epoch 12/30, Loss: 0.5879


In [74]:
# Create a dictionary to store both the adapter state_dict and the kwargs
save_dict = {
    'adapter_state_dict': trained_adapter.state_dict(),
    'adapter_kwargs': adapter_kwargs
}

# Save the combined dictionary
#torch.save(save_dict, '/content/Adapter')

In [75]:
# Save the adapter state_dict and kwargs to local files
torch.save(trained_adapter.state_dict(), '/content/Adapter/trained_adapter.pth')
torch.save(adapter_kwargs, '/content/Adapter/adapter_kwargs.pth')

In [85]:
# Function to encode query using the adapter
def encode_query(query, base_model, adapter):
    device = next(adapter.parameters()).device
    query_emb = base_model.encode(query, convert_to_tensor=True).to(device)
    adapted_query_emb = adapter(query_emb)
    return adapted_query_emb.cpu().detach().numpy()

In [80]:
import torch

# Load the saved model and kwargs
loaded_dict_adapter = torch.load('/content/Adapter/trained_adapter.pth')
loaded_dict_kwargs = torch.load('/content/Adapter/adapter_kwargs.pth')

# Recreate the adapter
loaded_adapter = LinearAdapter(base_model.get_sentence_embedding_dimension())  # Initialize with appropriate parameters

# Load the model's state_dict
loaded_adapter.load_state_dict(loaded_dict_adapter)  # Directly use the loaded state_dict

# Access the training parameters
training_params = loaded_dict_kwargs  # No need to access via key, it's already the dictionary

print("Adapter loaded successfully.")
print("Training parameters used:")
for key, value in training_params.items():
    print(f"{key}: {value}")


Adapter loaded successfully.
Training parameters used:
num_epochs: 1
batch_size: 32
learning_rate: 0.003
warmup_steps: 100
max_grad_norm: 1.0
margin: 1.0


<ipython-input-80-02ef5c38db66>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_dict_adapter = torch.load('/content/Adapter/trained_adapter.pth')
<ipython-input-80-0

In [86]:
def evaluate_adapter(validation_data, base_model, adapter, k=10):
    hit_rates = []
    reciprocal_ranks = []

    for _, row in validation_data.iterrows():  # Iterate over rows of the DataFrame
        question = row['question']
        ground_truth = row['context']
        answer = row['answer']

        # Generate embedding for the question
        question_embedding = encode_query(question, base_model, adapter)
        # Retrieve documents using the embedding
        retrieved_docs = retrieve_documents_embeddings(question_embedding, k)

        # Calculate metrics
        hr = hit_rate(retrieved_docs, ground_truth, k)
        rr = reciprocal_rank(retrieved_docs, ground_truth, k)

        hit_rates.append(hr)
        reciprocal_ranks.append(rr)

    # Calculate average metrics
    avg_hit_rate = np.mean(hit_rates)
    avg_reciprocal_rank = np.mean(reciprocal_ranks)

    return {
        'average_hit_rate': avg_hit_rate,
        'average_reciprocal_rank': avg_reciprocal_rank
    }

results = evaluate_adapter(test_data, base_model, trained_adapter, k=10)
print(f"Average Hit Rate @10: {results['average_hit_rate']}")
print(f"Mean Reciprocal Rank @10: {results['average_reciprocal_rank']}")

Average Hit Rate @10: 0.009569377990430622
Mean Reciprocal Rank @10: 0.0033492822966507173


In [82]:
import numpy as np
def validate_embedding_model(validation_data, base_model, k=10):
    hit_rates = []
    reciprocal_ranks = []

    for _, row in validation_data.iterrows():  # Iterate over rows of the DataFrame
        question = row['question']
        ground_truth = row['context']
        answer = row['answer']

        # Generate embedding for the question
        question_embedding = base_model.encode(question)
        #print(" ### Question embedding ### :",question)
        #print(" ### Ground Truth ### :",ground_truth)

        #print(" ### Answer ### :",answer)

        # Retrieve documents using the embedding
        retrieved_docs = retrieve_documents_embeddings(question_embedding, k)
        #print("  ### Retrieved Documents #### :")
        #for idx, doc in enumerate(retrieved_docs, 1):  # Start enumeration at 1
         #   print(f"\nDocument {idx}:\n{'-' * 40}\n{doc}\n{'-' * 40}")


        # Calculate metrics
        hr = hit_rate(retrieved_docs, ground_truth, k)
        rr = reciprocal_rank(retrieved_docs, ground_truth, k)

        hit_rates.append(hr)
        reciprocal_ranks.append(rr)

    # Calculate average metrics
    avg_hit_rate = np.mean(hit_rates)
    avg_reciprocal_rank = np.mean(reciprocal_ranks)

    return {
        'average_hit_rate': avg_hit_rate,
        'average_reciprocal_rank': avg_reciprocal_rank
    }

# Call the function with the corrected implementation
results = validate_embedding_model(test_data, base_model)
print(f"Average Hit Rate @10: {results['average_hit_rate']}")
print(f"Mean Reciprocal Rank @10: {results['average_reciprocal_rank']}")


Average Hit Rate @10: 0.5598086124401914
Mean Reciprocal Rank @10: 0.3997531708058024
